In [ ]:
import numpy as np
import pandas as pd

from pymc_marketing.mmm import MMM, GeometricAdstock, LogisticSaturation
from pymc_marketing.mmm.transformers import geometric_adstock, logistic_saturation
from pymc_marketing.prior import Prior
from mmm_eval import save_config
from mmm_eval.data import generate_data

from mmm_eval.utils import PyMCConfigRehydrator
from mmm_eval.cli.evaluate import load_config



In [2]:
def verify_hydration(config):
    from pymc_marketing.mmm.components.adstock import AdstockTransformation
    from pymc_marketing.mmm.components.saturation import SaturationTransformation
    
    class_map = {
        "adstock": AdstockTransformation,
        "saturation": SaturationTransformation,
    }

    for k, v in config["model_config"].items():
        if k in class_map:
            assert isinstance(v, class_map[k]), f"Expected {k} to be {class_map[k]}, got {type(v)}"

# Generate data

In [3]:
data = generate_data()
data.to_csv("data.csv", index=False)

X = data.drop(columns=["revenue","quantity"])
y = data["quantity"]

# Fit PyMC

In [ ]:
my_model = MMM(
    date_column="date_week" ,
    channel_columns=["channel_1","channel_2"],
    adstock=GeometricAdstock(l_max=4),
    saturation=LogisticSaturation()
)

my_model.fit(X=X, y=y, chains=4, target_accept=0.85)

# Save Config to JSON

Copy the args for MMM() and .fit() into dict structure. Loader will handle string representation.
Need to provide target column name b/c all columns will be in one CSV in the input data.

In [ ]:
model_args = {
    "date_column": "date_week",
    "channel_columns": ["channel_1", "channel_2"],
    "adstock": GeometricAdstock(l_max=4),
    "saturation": LogisticSaturation()
}

fit_args = {
    "X": X,  
    "y": y,   
    "chains": 4,
    "target_accept": 0.85,
}

target_column = "revenue"

string_config = save_config(model_args, fit_args, target_column=target_column, save_path="./", file_name="config")
string_config

# Rehydrate from saved config

In [ ]:
loaded_config = load_config("config.json")

# This should fail (still all strings)
verify_hydration(loaded_config)

In [31]:
rehydrated = PyMCConfigRehydrator(loaded_config.copy()).rehydrate_config()

# This should pass
verify_hydration(rehydrated)

# Fit a model with the input data + rehydrated config

In [32]:
import pandas as pd

df = pd.read_csv("data.csv")
df["date_week"] = pd.to_datetime(df["date_week"])

model_config = rehydrated["model_config"]
fit_config = rehydrated["fit_config"]
target_column = rehydrated["target_column"]

X = df.drop(columns=[target_column])
y = df[target_column]

In [ ]:
model_config

In [34]:
date_column = eval(model_config["date_column"])
channel_columns = list(eval(model_config["channel_columns"]))
adstock = model_config["adstock"]
saturation = model_config["saturation"]

m2 = MMM(
    date_column=date_column,
    channel_columns=channel_columns,
    adstock=adstock,
    saturation=saturation
)


In [ ]:
chains = int(eval(fit_config["chains"]))
target_accept = float(fit_config["target_accept"])

m2.fit(X=X, y=y, chains=chains, target_accept=target_accept)